# Introduction of Natural Language Processing(NLP)

In [50]:
import pandas as pd
import math
import re
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import *
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

## [Task 1] Bag-of-Words(BoW) and N-gram

<br />

I am going to remove special characters such as ! and 〜, and split words, then quantify/vectorize the following corpus by BoW(1-gram) and BoW(2-gram).

1: 今撮影中で〜す！


2: 今休憩中で〜す(^^)


3: 今日ドラマ撮影で〜す！


4: 今日、映画瞬公開で〜す！！！

#### BoW(1-gram)

In [2]:
# Example

vocabulary = ["I", "love", "this", "is", "the", "baseball"]
ms_kk_texts = ["I love baseball !!", "I love this !"]
texts_vec = [[1,1,0,0,0,1], [1,1,1,0,0,0]]

df_bow_1gram = pd.DataFrame(data=texts_vec, columns=vocabulary, index=ms_kk_texts)

In [3]:
df_bow_1gram

,I,love,this,is,the,baseball
I love baseball !!,1,1,0,0,0,1
I love this !,1,1,1,0,0,0


In [4]:
# Above four sentences

voc_1g = ["今", "撮影", "中", "です", "休憩", "今日", "ドラマ", "映画", "瞬", "公開"]
texts_1g = ["今撮影中で〜す！", "今休憩中で〜す(^^)", "今日ドラマ撮影で〜す！", "今日、映画瞬公開で〜す！！！"]
vec_1g = [[1,1,1,1,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0], [0,1,0,1,0,1,1,0,0,0], [0,0,0,1,0,1,0,1,1,1]]

df_bow_1g = pd.DataFrame(data=vec_1g, columns=voc_1g, index=texts_1g)

In [5]:
df_bow_1g

,今,撮影,中,です,休憩,今日,ドラマ,映画,瞬,公開
今撮影中で〜す！,1,1,1,1,0,0,0,0,0,0
今休憩中で〜す(^^),1,0,1,1,1,0,0,0,0,0
今日ドラマ撮影で〜す！,0,1,0,1,0,1,1,0,0,0
今日、映画瞬公開で〜す！！！,0,0,0,1,0,1,0,1,1,1


#### BoW(2-gram)

In [6]:
# Above four sentences

voc_2g = ["今撮影", "撮影中", "中です", "今休憩", "休憩中", "今日ドラマ", "ドラマ撮影", "撮影です", "今日映画", "映画瞬", "瞬公開", "公開です"]
texts_2g = ["今撮影中で〜す！", "今休憩中で〜す(^^)", "今日ドラマ撮影で〜す！", "今日、映画瞬公開で〜す！！！"]
vec_2g = [[1,1,1,0,0,0,0,0,0,0,0,0], [0,0,1,1,1,0,0,0,0,0,0,0], [0,0,0,0,0,1,1,1,0,0,0,0], [0,0,0,0,0,0,0,0,1,1,1,1]]

df_bow_2g = pd.DataFrame(data=vec_2g, columns=voc_2g, index=texts_2g)

In [7]:
df_bow_2g

,今撮影,撮影中,中です,今休憩,休憩中,今日ドラマ,ドラマ撮影,撮影です,今日映画,映画瞬,瞬公開,公開です
今撮影中で〜す！,1,1,1,0,0,0,0,0,0,0,0,0
今休憩中で〜す(^^),0,0,1,1,1,0,0,0,0,0,0,0
今日ドラマ撮影で〜す！,0,0,0,0,0,1,1,1,0,0,0,0
今日、映画瞬公開で〜す！！！,0,0,0,0,0,0,0,0,1,1,1,1


## [Task 2] TF-IDF

<br />

I will quantify/vectorize the corpus of the 1st task by TF-IDF.

#### Standard Formula of TF-IDF

Term Frequency(TF):

$$
tf(t,d) = \frac{n_{t,d}}{\sum_{s \in d}n_{s,d}}
$$

$n_{t,d}$ : Number of words "t" on a document "d"

$\sum_{s \in d}n_{s,d}$ : Sum of number of all words on a document "d"

Inverse Document Frequency(IDF):

$$
idf(t) = \log{\frac{N}{df(t)}}
$$

$N$ : Number of all documents

$df(t)$ : Number of documents that a word "t" appears

TF-IDF:

$$
tfidf(t, d) = tf(t, d) \times idf(t)
$$

In [8]:
# Above four sentences

original_texts = ["今撮影中で〜す！", "今休憩中で〜す(^^)", "今日ドラマ撮影で〜す！", "今日、映画瞬公開で〜す！！！"]

voc_tf_idf = ["今", "撮影", "中", "です", "休憩", "今日", "ドラマ", "映画", "瞬", "公開"]
texts_tf_idf = ["今撮影中です", "今休憩中です", "今日ドラマ撮影です", "今日映画瞬公開です"]
vec_tf_idf = [[1,1,1,1,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0], [0,1,0,1,0,1,1,0,0,0], [0,0,0,1,0,1,0,1,1,1]]

In [9]:
# Compute TF

tf_list = []
for i in range(len(texts_tf_idf)):
    tf = []
    for j in range(len(voc_tf_idf)):
        tf.append(texts_tf_idf[i].count(voc_tf_idf[j]) / vec_tf_idf[i].count(1))
    tf_list.append(tf)

In [10]:
tf_list

[[0.25, 0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.25, 0.0, 0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.25, 0.25, 0.0, 0.25, 0.0, 0.25, 0.25, 0.0, 0.0, 0.0],
 [0.2, 0.0, 0.0, 0.2, 0.0, 0.2, 0.0, 0.2, 0.2, 0.2]]

In [11]:
# Compute IDF

idf_list = []
for j in range(len(vec_tf_idf[0])):
    count = 0
    for i in range(len(texts_tf_idf)):
        if vec_tf_idf[i][j] == 1:
            count += 1
    idf = math.log2(len(texts_tf_idf) / count)
    idf_list.append(idf)

In [12]:
idf_list

[1.0, 1.0, 1.0, 0.0, 2.0, 1.0, 2.0, 2.0, 2.0, 2.0]

In [13]:
# Compute TF-IDF

tf_idf_list = []
for i in range(len(tf_list)):
    tf_idf = []
    for j in range(len(idf_list)):
        tf_idf.append(tf_list[i][j] * idf_list[j])
    tf_idf_list.append(tf_idf)

In [14]:
tf_idf_list

[[0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.25, 0.0, 0.25, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.25, 0.25, 0.0, 0.0, 0.0, 0.25, 0.5, 0.0, 0.0, 0.0],
 [0.2, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.4, 0.4, 0.4]]

In [15]:
# Dataframe

df_tf_idf = pd.DataFrame(data=tf_idf_list, columns=voc_tf_idf, index=original_texts)

In [16]:
df_tf_idf

,今,撮影,中,です,休憩,今日,ドラマ,映画,瞬,公開
今撮影中で〜す！,0.25,0.25,0.25,0.0,0.0,0.00,0.0,0.0,0.0,0.0
今休憩中で〜す(^^),0.25,0.00,0.25,0.0,0.5,0.00,0.0,0.0,0.0,0.0
今日ドラマ撮影で〜す！,0.25,0.25,0.00,0.0,0.0,0.25,0.5,0.0,0.0,0.0
今日、映画瞬公開で〜す！！！,0.20,0.00,0.00,0.0,0.0,0.20,0.0,0.4,0.4,0.4


## [Task 3] Text Cleaning

<br />

I am going to remove urls, 【◯◯】, special characters including line breaks and emojis.

In [144]:
# Target
text = "<!everyone> *【スペシャル特典】有償のRubyMineやPyCharmの `6ヶ月間100%OFFクーポン` をご希望者の方先着100名様に贈呈いたします！*\n\nこの度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。\n\n```\nRubyMine\n<https://www.jetbrains.com/ruby/>\n\nPyCharm\n<https://www.jetbrains.com/pycharm/>\n```\n\n「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。\n\n期限は、 *`2019年3月20日（水）22:00まで`* とさせていただきます。\nふるってのご希望をお待ちしております！ :smile:"

# <◯◯>
text = re.sub(r"<(.+)>", "", text)
# 【◯◯】
text = re.sub(r"【(.+)】", "", text)
# Urls
text = re.sub(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+", "", text)
# Emojis
text = re.sub(r":([a-xA-Z0-9_]+):", "", text)
# Special characters
text = re.sub(r"[\n\*`\s]", "", text)

In [145]:
text

'有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！'

In [146]:
# Check

correct_text = "有償のRubyMineやPyCharmの6ヶ月間100%OFFクーポンをご希望者の方先着100名様に贈呈いたします！この度、RubyMineやPyCharmのメーカーであるJetBrains社へのクーポンコードの提供交渉が実り、100クーポンをいただくことができました。RubyMinePyCharm「ご希望の方は、手を挙げて！」方式で、ご希望の方はこの投稿の手あげスタンプをクリックしてください。期限は、2019年3月20日（水）22:00までとさせていただきます。ふるってのご希望をお待ちしております！"

text == correct_text

True

## [Task 4] Morphological Analysis

<br />

There are tools for morphological analysis, like Mecab and Janome. In this task, I will use Janome.


https://mocobeta.github.io/janome/

<br />

I will do morphological analysis about the text that I did text cleaning on the previous task to extract nouns and verbs by Janome.

In [60]:
t = Tokenizer()
for token in t.tokenize(text):
    print(token)

有償	名詞,一般,*,*,*,*,有償,ユウショウ,ユーショー
の	助詞,連体化,*,*,*,*,の,ノ,ノ
RubyMine	名詞,一般,*,*,*,*,RubyMine,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
6	名詞,数,*,*,*,*,6,*,*
ヶ月	名詞,接尾,助数詞,*,*,*,ヶ月,カゲツ,カゲツ
間	名詞,接尾,一般,*,*,*,間,カン,カン
100	名詞,数,*,*,*,*,100,*,*
%	名詞,サ変接続,*,*,*,*,%,*,*
OFF	名詞,一般,*,*,*,*,OFF,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
ご	接頭詞,名詞接続,*,*,*,*,ご,ゴ,ゴ
希望	名詞,サ変接続,*,*,*,*,希望,キボウ,キボー
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
方	名詞,非自立,一般,*,*,*,方,ホウ,ホー
先着	名詞,サ変接続,*,*,*,*,先着,センチャク,センチャク
100	名詞,数,*,*,*,*,100,*,*
名	名詞,接尾,助数詞,*,*,*,名,メイ,メイ
様	名詞,接尾,人名,*,*,*,様,サマ,サマ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
贈呈	名詞,サ変接続,*,*,*,*,贈呈,ゾウテイ,ゾーテイ
いたし	動詞,非自立,*,*,五段・サ行,連用形,いたす,イタシ,イタシ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
！	記号,一般,*,*,*,*,！,！,！
この	連体詞,*,*,*,*,*,この,コノ,コノ
度	名詞,非自立,副詞可能,*,*,*,度,タビ,タビ
、	記号,読点,*,*,*,*,、,、,、
RubyMine	名詞,固有名詞,組織,*,*,*,RubyMine,*,*
や	助詞,並立助詞,*,*,*,*,や,ヤ,ヤ
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
メーカー	名詞,一般,*,*,*,*,メーカー,メーカー,メーカー
で	助

In [59]:
# Nouns

token_filters = [POSKeepFilter("名詞")]
a = Analyzer(token_filters=token_filters)
for token in a.analyze(text):
    print(token)

有償	名詞,一般,*,*,*,*,有償,ユウショウ,ユーショー
RubyMine	名詞,一般,*,*,*,*,RubyMine,*,*
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
6	名詞,数,*,*,*,*,6,*,*
ヶ月	名詞,接尾,助数詞,*,*,*,ヶ月,カゲツ,カゲツ
間	名詞,接尾,一般,*,*,*,間,カン,カン
100	名詞,数,*,*,*,*,100,*,*
%	名詞,サ変接続,*,*,*,*,%,*,*
OFF	名詞,一般,*,*,*,*,OFF,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
希望	名詞,サ変接続,*,*,*,*,希望,キボウ,キボー
者	名詞,接尾,一般,*,*,*,者,シャ,シャ
方	名詞,非自立,一般,*,*,*,方,ホウ,ホー
先着	名詞,サ変接続,*,*,*,*,先着,センチャク,センチャク
100	名詞,数,*,*,*,*,100,*,*
名	名詞,接尾,助数詞,*,*,*,名,メイ,メイ
様	名詞,接尾,人名,*,*,*,様,サマ,サマ
贈呈	名詞,サ変接続,*,*,*,*,贈呈,ゾウテイ,ゾーテイ
度	名詞,非自立,副詞可能,*,*,*,度,タビ,タビ
RubyMine	名詞,固有名詞,組織,*,*,*,RubyMine,*,*
PyCharm	名詞,一般,*,*,*,*,PyCharm,*,*
メーカー	名詞,一般,*,*,*,*,メーカー,メーカー,メーカー
JetBrains	名詞,一般,*,*,*,*,JetBrains,*,*
社	名詞,接尾,一般,*,*,*,社,シャ,シャ
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
コード	名詞,一般,*,*,*,*,コード,コード,コード
提供	名詞,サ変接続,*,*,*,*,提供,テイキョウ,テイキョー
交渉	名詞,サ変接続,*,*,*,*,交渉,コウショウ,コーショー
100	名詞,数,*,*,*,*,100,*,*
クーポン	名詞,一般,*,*,*,*,クーポン,クーポン,クーポン
こと	名詞,非自立,一般,*,*,*,こと,コト,コト
RubyMinePyCharm	名詞,一般,*,*,*,*,RubyMinePyCharm,*,*
希望	名詞,サ変接続,*,*,*,*,希望,キ

In [61]:
# Verbs

token_filters = [POSKeepFilter("動詞")]
a = Analyzer(token_filters=token_filters)
for token in a.analyze(text):
    print(token)

いたし	動詞,非自立,*,*,五段・サ行,連用形,いたす,イタシ,イタシ
実り	動詞,自立,*,*,五段・ラ行,連用形,実る,ミノリ,ミノリ
いただく	動詞,自立,*,*,五段・カ行イ音便,基本形,いただく,イタダク,イタダク
でき	動詞,自立,*,*,一段,連用形,できる,デキ,デキ
挙げ	動詞,自立,*,*,一段,連用形,挙げる,アゲ,アゲ
あげ	動詞,自立,*,*,一段,連用形,あげる,アゲ,アゲ
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
ください	動詞,非自立,*,*,五段・ラ行特殊,命令ｉ,くださる,クダサイ,クダサイ
さ	動詞,自立,*,*,サ変・スル,未然レル接続,する,サ,サ
せ	動詞,接尾,*,*,一段,連用形,せる,セ,セ
いただき	動詞,非自立,*,*,五段・カ行イ音便,連用形,いただく,イタダキ,イタダキ
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
おり	動詞,非自立,*,*,五段・ラ行,連用形,おる,オリ,オリ


## [Task 5] Analyze News

<br />

I will analyze some news of "livedoor" from various viewpoints.


http://news.livedoor.com/

In [64]:
# Show an explanation of the "livedoor" news

cat text/README.txt

livedoor ニュースコーパス


概要
-----------------
本コーパスは、NHN Japan株式会社が運営する「livedoor ニュース」のうち、下記のクリエイティブ・コモンズ
ライセンスが適用されるニュース記事を収集し、可能な限りHTMLタグを取り除いて作成したものです。

- トピックニュース
  http://news.livedoor.com/category/vender/news/
- Sports Watch
  http://news.livedoor.com/category/vender/208/
- ITライフハック
  http://news.livedoor.com/category/vender/223/
- 家電チャンネル
  http://news.livedoor.com/category/vender/kadench/
- MOVIE ENTER
  http://news.livedoor.com/category/vender/movie_enter/
- 独女通信
  http://news.livedoor.com/category/vender/90/
- エスマックス
  http://news.livedoor.com/category/vender/smax/
- livedoor HOMME
  http://news.livedoor.com/category/vender/homme/
- Peachy
  http://news.livedoor.com/category/vender/ldgirls/

収集時期：2012年9月上旬


ライセンス
-----------------
各記事ファイルにはクリエイティブ・コモンズライセンス「表示 - 改変禁止」
（http://creativecommons.org/licenses/by-nd/2.1/jp/）が適用されます。
クレジット表示についてはニュースカテゴリにより異なるため、サブディレクトリにある
それぞれの LICENSE.txt をご覧ください。

livedoor はNHN Japan株式会社の登録商標です。


記事ファイルのフォーマット
-----------------
記事ファイルは以下のフォーマットにしたが

In [7]:
# Read files
bin_data = load_files('./text', encoding='utf-8')

documents = bin_data.data

# Get labels while will not use them this time.
targets = bin_data.target

In [72]:
# Check

print(documents[0])

http://news.livedoor.com/article/detail/4931238/
2010-08-08T10:00:00+0900
NY名物イベントが日本でも！名店グルメを気軽に楽しむ
ニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。
　
　実行委員長には、学校法人服部学園、服部栄養専門学校 理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LA BETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。

参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストラン タテル ヨシノ 汐留」や、日本料理の名門「つきぢ田村」、「金田中 庵」、「赤坂璃宮」に「mikuni MARUNOUCHI」など、日本を代表するレストランがずらり。
　イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。

　予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？

JAPAN RESTAURANT WEEK 2010 -公式サイト



In [99]:
documents[0]

'http://news.livedoor.com/article/detail/4931238/\n2010-08-08T10:00:00+0900\nNY名物イベントが日本でも！名店グルメを気軽に楽しむ\nニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。\n\u3000\n\u3000実行委員長には、学校法人服部学園、服部栄養専門学校 理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LA BETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。\n\n参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストラン タテル ヨシノ 汐留」や、日本料理の名門「つきぢ田村」、「金田中 庵」、「赤坂璃宮」に「mikuni MARUNOUCHI」など、日本を代表するレストランがずらり。\n\u3000イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。\n\n\u3000予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？\n\nJAPAN RESTAURANT WEEK 2010 -公式サイト\n'

In [73]:
targets

array([5, 8, 7, ..., 1, 3, 3])

### Analyze News by Counting Vocabularies

In [83]:
token_filters1 = [TokenCountFilter()]
a1 = Analyzer(token_filters=token_filters1)
dic = {}
for doc in documents:
    for k, v in a1.analyze(doc):
        if k not in dic:
            dic[k] = v
        else:
            dic[k] += v

In [38]:
# print(dic)

    - Heavy to show

In [97]:
# Sort

dic_sorted = sorted(dic.items(), key=lambda x: x[1], reverse=True)

In [39]:
# dic_sorted[0]

    - Heavy to show

### Text Cleaning

In [8]:
doc_clean = []
for doc in documents:
    # Remove urls
    BAD_SYMBOL = re.compile(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+")
    doc = re.sub(BAD_SYMBOL, "", doc)
    
    # Remove dates
    BAD_SYMBOL = re.compile(r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\+\d{4}")
    doc = re.sub(BAD_SYMBOL, "", doc)
    
    # Remove special characters
    BAD_SYMBOL = re.compile(r"[\n\*`\s:：「」、,。\.・/／\-\(\)（）!！『』？\?【】”“■\+＋〜~※&＆×％|●◯＜＞＝=★→]+")
    doc = re.sub(BAD_SYMBOL, "", doc)
    
    # Replace all numbers to "NUMBER"
    BAD_SYMBOL = re.compile(r"\d+")
    doc = re.sub(BAD_SYMBOL, "NUMBER", doc)
    
    doc_clean.append(doc)

In [4]:
doc_clean[0]

'NY名物イベントが日本でも名店グルメを気軽に楽しむニューヨークでNUMBER年続いている食の祭典レストランウィークその日本版がダイナーズクラブ特別協賛のもとNUMBER月NUMBER日よりスタートNUMBER月NUMBER日までの期間中青山六本木丸の内銀座横浜のエリアからラグジュアリーレストランNUMBER店舗がこのイベントのために特別用意したランチメニューNUMBER円税サ別ディナーNUMBER円税サ別を気軽に楽しめるとっておきのイベントです実行委員長には学校法人服部学園服部栄養専門学校理事長校長であり医学博士でもある服部幸應氏を迎え実行委員に石田純一さんLABETTOLAオーナーシェフ落合務氏フードアナリスト協会会長高賀右近氏つきぢ田村三代目田村隆氏にそして放送作家脚本家の小山薫堂さんなど食のスペシャリストたちが勢揃い参加レストランにはミシュランのフランス版東京版ともに星を獲得している吉野建シェフのレストランタテルヨシノ汐留や日本料理の名門つきぢ田村金田中庵赤坂璃宮にmikuniMARUNOUCHIなど日本を代表するレストランがずらりイベント期間のNUMBER月NUMBER日までは特別協賛のダイナーズクラブカード会員またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますがその後は誰でも参加できるので日程のチェックは必須予約方法は必ず事前に各店舗に問合せを行いジャパンレストランウィークNUMBERでの予約であることを伝えればOK憧れていたレストランの料理をリーズナブルにいただけるチャンスです極上の味とラグジュアリーな空間を満喫そんな幸せを実感できるジャパンレストランウィークNUMBERにぜひ参加しててみてはいかがですかJAPANRESTAURANTWEEKNUMBER公式サイト'

### Vectorize by BoW and TF-IDF

In [11]:
# BoW (1-gram)

# Split documents into each word
t_wakati = Tokenizer(wakati=True)

news_voc = []
for doc in doc_clean:
    voc = t_wakati.tokenize(doc)
    news_voc.append(voc)

In [35]:
# Check

news_voc[0][0]

'NY'

In [13]:
# Unite all lists to a single list

united_voc = []
for i in range(len(news_voc)):
    for j in range(len(news_voc[i])):
        united_voc.append(news_voc[i][j])

In [14]:
# Check

print(united_voc[:10])
print(united_voc[0][0])

['NY', '名物', 'イベント', 'が', '日本', 'で', 'も', '名店', 'グルメ', 'を']
N


In [32]:
# Create a list of vectors

news_vec = [[] for i in range(len(doc_clean))]

for i in range(len(doc_clean)):
    for voc in united_voc:
        if voc in doc_clean[i]:
            news_vec[i].append(1)
        else:
            news_vec[i].append(0)

KeyboardInterrupt: 

    - The processing wasted too much time, so I gave up completing it and running the following 3 cells.

In [1]:
# # Check

# news_vec[0]

In [2]:
# Make data a dataframe

# df_news_bow = pd.DataFrame(data=news_vec, columns=united_voc, index=doc_clean)

In [3]:
# Show

# df_news_bow

In [27]:
# TF-IDF

space_doc = []
for i in range(len(news_voc)):
    doc = ""
    for j in range(len(news_voc[i])):
        doc += news_voc[i][j] + " "
    space_doc.append(doc)

In [28]:
# Initialize
vectorizer = TfidfVectorizer()

# Fit
X = vectorizer.fit_transform(space_doc)

In [36]:
# Show index

vectorizer.vocabulary_["ny"]

8507

In [31]:
# Get TF-IDF values

tf_idf_array = X.toarray()

In [48]:
# Check

tf_idf_array[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [33]:
sum(tf_idf_array[0])

10.362136716343215

    - Not 0, so it seems like the processing worked well.

In [64]:
# Create a definition to output a piece of news whose cosine similarity has the highest correlation with a piece of news selected

def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))


def closest_cos_sim(X, y):
    """
    X : All news except for y
    y : Single news selected
    """
    
    cos_similarity = 0
    index = -1
    for i in range(len(X)):
        a = cos_sim(X[i], y)
        if a == 1:
            pass
        elif a > cos_similarity:
            cos_similarity = a
            index = i
        else:
            pass
    return cos_similarity, index

#### 1st Validation

In [65]:
# Validate the definition

cos_sim1, index1 = closest_cos_sim(tf_idf_array, tf_idf_array[0])

In [66]:
print(cos_sim1)
print(index1)

0.1617431820331787
3548


In [81]:
# News selected

print(documents[0])

http://news.livedoor.com/article/detail/4931238/
2010-08-08T10:00:00+0900
NY名物イベントが日本でも！名店グルメを気軽に楽しむ
ニューヨークで20年続いている食の祭典「レストラン・ウィーク」。その日本版がダイナーズクラブ特別協賛のもと7月30日よりスタート。8月31日までの期間中、青山・六本木、丸の内、銀座、横浜のエリアから、ラグジュアリーレストラン81店舗がこのイベントのために特別用意したランチメニュー2010円（税・サ別）、ディナー5000円（税・サ別）を気軽に楽しめる、とっておきのイベントです。
　
　実行委員長には、学校法人服部学園、服部栄養専門学校 理事長・校長であり医学博士でもある服部幸應氏を迎え、実行委員に石田純一さん、LA BETTOLAオーナーシェフ落合務氏、フードアナリスト協会会長、高賀右近氏、つきぢ田村三代目、田村隆氏に、そして放送作家・脚本家の小山薫堂さんなど、食のスペシャリストたちが勢揃い。

参加レストランには、ミシュランのフランス版、東京版ともに星を獲得している吉野建シェフの「レストラン タテル ヨシノ 汐留」や、日本料理の名門「つきぢ田村」、「金田中 庵」、「赤坂璃宮」に「mikuni MARUNOUCHI」など、日本を代表するレストランがずらり。
　イベント期間の〜8月19日までは、特別協賛のダイナーズクラブカード会員、またはシティバンクに口座を持つシティゴールドメンバーが楽しめる先行期間となりますが、その後は誰でも参加できるので、日程のチェックは必須。

　予約方法は必ず事前に、各店舗に問合せを行い「ジャパンレストラン・ウィーク2010」での予約であることを伝えればOK！憧れていたレストランの料理をリーズナブルにいただけるチャンスです！極上の味とラグジュアリーな空間を満喫。そんな幸せを実感できる「ジャパンレストラン・ウィーク2010」にぜひ参加しててみてはいかがですか？

JAPAN RESTAURANT WEEK 2010 -公式サイト



In [82]:
# News whose cosine similarity has the highest correlation with a piece of news selected

print(documents[index1])

http://news.livedoor.com/article/detail/5985275/
2011-11-01T12:27:00+0900
ロンブー田村淳のTV業界論にネットユーザー激怒「嫌ならやめろ」
先日、ロンドンブーツ1号2号の田村淳と、ジャーナリスト・津田大介による「テレビを巡るこれからのメディア論」の対談が「TV Bros.」10月29日発売号（東京ニュース通信社刊）に掲載された。

その対談の中で、田村は
「あまりに古い感覚でやっている人と仕事をするのはすごいストレスですよ。なんでこれが面白いってわからないんですか？って思うことがしょっちゅうある」
「どうしても自分のやりたいことができなくなって、もうテレビは無理だと思ったらスパーンと全部やめる覚悟はありますよ」
などと衝撃発言を行った。

すると、この田村の発言を受けて、ネットの掲示板では田村バッシングが加熱、
「なんでわかんないの？って感覚でみんな番組作ってるから視聴者が離れていってるんじゃないですかね」
「お前がつまらないからだよ」
「嫌ならやめろ」
といった批判が殺到している。

最近見る田村の発言は、ネットユーザーの怒りを買うものばかりだ。以前、ツイッター上でも
「視聴者に、嫌ならテレビ見なければいいというのなら、あなた方ネット批判芸能人も、ネットをしなければよいのでは？」
と問いかけられたことに対し、
「ネチネチネチネチうるせーなぁ…って周りに言われない？」と発言、ネットユーザーを激怒させたこともあった。

田村の尖った発言の数々は、ネットユーザーの怒りを増長させる傾向にあるだけに、今後の成り行きが心配される。

【関連記事】
・田村淳「刺激的な番組をつくれないとそれは衰退」。津田大介と“テレビを巡るメディア論”で対談

【関連情報】
・田村淳、TV業界を語る「古い感覚でやっている人と仕事をするのはすごいストレス。なんでこれが面白いってわからないの？」

・ロンブー田村淳がツイッター民の正論に激怒「ネチネチうるせぇ」　こんな返し方しかできない芸人って



    - It seems like those news are not correlated strongly.

#### 2nd

In [71]:
# Validate the definition

cos_sim2, index2 = closest_cos_sim(tf_idf_array, tf_idf_array[1])

In [72]:
print(cos_sim2)
print(index2)

0.3311325289270498
2872


In [83]:
# News selected

print(documents[1])

http://news.livedoor.com/article/detail/6655079/
2012-06-13T19:25:00+0900
小沢一郎氏の妻が支援者に離婚を報告。「週刊文春」報じる
13日、Web版「週刊文春」は、民主党の元代表・小沢一郎氏の妻が、支援者宛に離婚したことを伝える手紙を送ったと報じ、ツイッターやネット掲示板で大きな話題になっている。

記事によると、その手紙は「小沢は放射能が怖くて秘書と一緒に逃げだしました」「隠し子が発覚した際、小沢元代表は和子夫人に謝るどころか、『いつでも離婚してやる』と言い放ち、和子夫人は一時は自殺まで考えた」という小沢氏の支持者にとってはショッキングな内容となっている。

ツイッターやネット掲示板では、「これが小沢一郎の本性か」「こりゃすごいな。てか、がっかりだなあ」などと、手紙の内容に驚く声が相次ぎ、その他にも「小沢潰しですかね?」「元秘書が交番で暴れて逮捕された件といい、小沢氏を潰したい何者かによる工作活動では?」といった声が挙がっていた。

【関連情報】
・小沢一郎夫人が支援者に「離婚しました」（週刊文春）



In [84]:
# News whose cosine similarity has the highest correlation with a piece of news selected

print(documents[index2])

http://news.livedoor.com/article/detail/5633553/
2011-06-14T16:29:00+0900
クルマ色話 vol.1【ベンツCLS】 コレは斎藤佑樹クンが乗らないとダメでしょ!?
20世紀的バリューを思いっきり引きずりつつ未来を目指すアラフォー自動車ジャーナリスト、小沢コージと業界の由美かおる!?　こと奇跡の超美人カーライフエッセイスト、吉田由美がお送りするお気楽かつキレキレな自動車トーク「クルマ色話」。一発目は昔の匂いを感じさせる、あの高級4ドアクーペ「メルセデス・ベンツCLS」だ！

◆プロフィール
小沢コージ
バブル期にソアラ、プレリュードを横目で眺め、トレンディドラマを見、合コンしまくったスーパーカーブームど真ん中イケメン自動車ジャーナリスト。現在『BESTCAR』『ENGINE』『DIME』誌ほか『webCG』『日経TRENDY』『carview』などwebでも大活躍！

吉田由美
当時、ミスコン荒らしとして20コ以上のミスの肩書きを持ち、某専門誌「○×のすべて」に助手席モデルとして君臨。「経歴は編集長より長いかも（笑）」（本人談）という業界きっての謎の美女。今はカーライフエッセイストの名のもと「ブログの女王」として自動車メディアはもちろんファッション誌に無くてはならない存在となっている。



最近、欲望ギラギラの男達が少なくなった……

吉田：アタシ、このクルマ好きだなぁなんとなく。
小沢コージ（以下、小沢）：なんとなくじゃなくってちゃんと理由言ってよ。ハタチの小娘じゃないんだからさ。
吉田：いきなり感じ悪ぅ〜。彼女とケンカでもした？
小沢：そんな相手、今いないよ。そんな余裕ナシ。で、その理由は。
吉田：バブルの匂いがするじゃない。妙に落ち着く（笑）。
小沢：理屈ないけど、それわかる。確かに欲望ギラギラだよね。CLSって6年前に出た元祖４ドアクーペで、一見クーペなのに実はセダンで「既婚だけど独身に見せたい」って願望まる出し（笑）。なおかつ旧 型はリアからはクラシックなジャガーEタイプそっくりで、新作はそれをマッチョかつ快適にした感じ。昔、日本にも「4ドアクーペ」のカリーナEDって あったけど、それよりずっとスケールデカいよ。中身はその時代のベンツＥクラスで味は保証付きだしさ。

「カリーナED」初代

    - Both of news are about a celebrity whose family name is "小沢". They are not a same person, though.
    
    - Those news are gossip articles.

## [Task 6] Sentiment Analysis